# Tutorial 2: Automated Lineament Extraction
In this tutorial we demosntrate how to extract lineament from gridded magnetic data in a semi-automated fashion. The goal is not to replace human interpretation, but rather to speed up the work by providing a "first guess" using machine vision algorithms provided by the `scikit-learn` package.

In [1]:
from shutil import copyfile
import numpy as np
from geoh5io.workspace import Workspace
from geoh5io.objects import Grid2D, Curve

Load pre-computed Filters
-------

For this example we have pre-computed the reduce-to-pole (RTP) magnetic data from Mount Dore, Australia.

![Workspace](./images/Tutorial2_MtDore_Tilt.png)

We first load the data from the provided geoh5 file.

In [6]:
# Lets create a working copy
copyfile("assets/MtDore_TMI.geoh5", "assets/MtDore_TMI_temp.geoh5")

# Load the project
workspace = Workspace("assets/MtDore_TMI_temp.geoh5")

# Load the grid
grid = workspace.get_entity("TMI")[0]
print(grid.get_data_list())

['TMI', 'TMI rtp filtered', 'Visual Parameters']


# Extract spatial data
--------------
In order to estimate the position and orientation of magnetic contacts, we need to 
- Extract edges using the Canny algorithm from `skimage.feature`.
- Detect lines using the Hough tranform from `skimage.transform`

![Workspace](./images/Tutorial2_MtDore_HoughLine.png)



In [9]:
from skimage.feature import canny
from skimage.transform import probabilistic_hough_line

X = grid.centroids[:, 0].reshape(grid.shape, order="F")
Y = grid.centroids[:, 1].reshape(grid.shape, order="F")
rtp = grid.get_data('TMI rtp filtered')[0].values.reshape(grid.shape, order="F")

# Parameters controling the edge detection 
edges = canny(rtp.T, 0.001, 0.01, 0.01)

# Parameters controling the line detection
lines = probabilistic_hough_line(edges, threshold=10, line_length=10,
                                 line_gap=3)

# Extract the line features
xy = []
cells = []
count = 0
for line in lines:
    p0, p1 = line

    xy.append(np.c_[X[p0[0], 0], Y[0, p0[1]] ,0])
    xy.append(np.c_[X[p1[0], 0], Y[0, p1[1]], 0])
    
    cells.append(np.c_[count, count+1].astype("uint32"))
    
    count += 2

# Save the result to geoh5
curve = Curve.create(
    workspace,
    name="HoughLines",
    vertices=np.vstack(xy),
    cells=np.vstack(cells)
)